In [ ]:
import sagemaker
import boto3 
from sagemaker.image_uris import retrieve
from sagemaker.pytorch import PyTorch, PyTorchModel
import os 
from PIL import Image
from torchvision import transforms
import json
# from sagemaker.inputs import TrainingInput

In [ ]:
sm_boto3 = boto3.client("sagemaker")

sess = sagemaker.Session()

region = sess.boto_session.region_name

# role = get_execution_role()

In [ ]:
bucket = os.getenv("BUCKET")
role_arn = os.getenv("ROLE_ARN")

In [ ]:

if not bucket  and role_arn:
    
    raise ValueError("Las variables de entorno de SAGEMAKER no estan configuradas.")

else: print("Variables cargados exitosamente")

# Train

In [ ]:
hyperparameters={
        'epochs': 8 ,
        'batch_size': 32,
        'learning_rate': 0.001
    }

In [ ]:
estimator = PyTorch(
    entry_point='resnet50.py',  
    role=role_arn,
    instance_count=1,  
    instance_type='ml.g4dn.xlarge', 
    framework_version='1.13.0', 
    py_version='py38',  
    hyperparameters=hyperparameters,
    output_path= bucket + "output",  
    sagemaker_session=sess
    use_spot_intances=True,
    max_wait=7200,
    max_run=3600
)

In [ ]:
training_data = bucket + "Augmented Dataset/"
estimator.fit(training_data)


# Deploy


In [ ]:
model = PyTorchModel(
    model_data=estimator.model_data,
    role=role_arn,
    entry_point="inference.py",
    framework_version='1.13',
    py_version='py38',
    source_dir='code',
)

In [ ]:
# Desplegar el modelo
endpoint_name = 'dragon-fruit-endpoint'

predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.g4dn.xlarge', 
    endpoint_name=endpoint_name
)


# Inference

In [ ]:

sagemaker_runtime = boto3.client('sagemaker-runtime')

In [ ]:
image_path = 'Augmented_Dataset/normal/Fresh_Dragon_Augmented_Data0003.jpg'
image = Image.open(image_path)
image = image.resize((224, 224))  


In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),  # Convertir a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalización estándar
])


In [ ]:
input_tensor = transform(image).unsqueeze(0) 
payload = json.dumps(input_tensor.numpy().tolist())

In [ ]:
# Hacer la predicción llamando al endpoint
response = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",  
)

In [ ]:
result = json.loads(response['Body'].read().decode())
print(f"Resultado de la predicción: {result}")

In [ ]:
predictor.delete_endpoint()